Type

In [6]:
# Age : Integer (Années)
# Attrition : Boolean
# BusinessTravel : Categorical (Non-Travel, Travel_Rarely, Travel_Frequently)
# DistanceFromHome : Integer (km)
# Education : Categorical (Avant College, College, Bachelor, Master, PhD)
# EducationField : Categorical (Domaines d’étude)
# EmployeeId : Integer (Identifiant unique)
# Gender : Boolean (Homme/Femme)
# JobLevel : Integer (1-5, niveau hiérarchique)
# JobRole : Categorical (Intitulé des postes)
# MaritalStatus : Categorical (Single, Married, Divorced)
# MonthlyIncome : Float (Salaire mensuel)
# NumCompaniesWorked : Integer (Nombre d’entreprises)
# PercentSalaryHike : Float (Pourcentage)
# StockOptionLevel : Integer (Niveau d’actions : 0, 1, 2, 3)
# TotalWorkingYears : Integer (Années)
# TrainingTimesLastYear : Integer (Jours)
# YearsAtCompany : Integer (Années)
# YearsSinceLastPromotion : Integer (Années)
# YearsWithCurrentManager : Integer (Années)
# JobInvolvement : Integer (1-4, Niveau d’implication)
# PerformanceRating : Integer (1-4, Niveau de performance)
# EnvironmentSatisfaction : Integer ou Null (1-4, Niveau de satisfaction, “NA” pour non-réponse)
# JobSatisfaction : Integer ou Null (1-4, Niveau de satisfaction, “NA” pour non-réponse)
# WorkLifeBalance : Integer ou Null (1-4, Niveau de satisfaction, “NA” pour non-réponse)

In [7]:
# Import des bibliothèques principales
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
import streamlit as st


In [8]:
# Charger les données
hr_data = pd.read_csv('./data/general_data.csv')
survey_data = pd.read_csv('./data/employee_survey_data.csv')
manager_data = pd.read_csv('./data/manager_survey_data.csv')
in_time_data = pd.read_csv('./data/in_time.csv')
out_time_data = pd.read_csv('./data/out_time.csv')

# Aperçu des données
#print(in_time_data.head())
#print(out_time_data.head())

# Voir les donnees manquantes
#print(hr_data.isnull().sum())
#print(survey_data.isnull().sum())
#print(manager_data.isnull().sum())

# Supprimer la colonne EmployeeCount, StandardHours, Over18
hr_data.drop(['EmployeeCount', 'StandardHours', 'Over18'], axis=1, inplace=True)

# Supprimer les lignes pour NumCompaniesWorked vide 
hr_data.dropna(subset=['NumCompaniesWorked'], inplace=True)

# Supprimer les lignes pour TotalWorkingYears vide
hr_data.dropna(subset=['TotalWorkingYears'], inplace=True)

In [9]:
# Renommer la première colonne pour qu'elle devienne "EmployeeID" dans in_time et out_time
in_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)
out_time_data.rename(columns={"Unnamed: 0": 'EmployeeID'}, inplace=True)
# Definir le format en date de toutes les colonnes (type) pour in_time et out_time sauf la colonne EmployeeID
for col in in_time_data.columns[1:]:
    in_time_data[col] = pd.to_datetime(in_time_data[col])

for col in out_time_data.columns[1:]:
    out_time_data[col] = pd.to_datetime(out_time_data[col])


# Fusionner les données de chaque fichier
hr_data = hr_data.merge(survey_data, on='EmployeeID')
hr_data = hr_data.merge(manager_data, on='EmployeeID')

#hr_data = hr_data.merge(in_time_data, on='EmployeeID')
#hr_data = hr_data.merge(out_time_data, on='EmployeeID')

# Remplacement des valeurs manquantes par la médiane arrondie
hr_data['EnvironmentSatisfaction'].fillna(hr_data['EnvironmentSatisfaction'].median().round(), inplace=True)
hr_data['JobSatisfaction'].fillna(hr_data['JobSatisfaction'].median().round(), inplace=True)
hr_data['WorkLifeBalance'].fillna(hr_data['WorkLifeBalance'].median().round(), inplace=True)


# Definir le format de chaque colonne (type)
hr_data['Age'] = hr_data['Age'].astype(int)
hr_data['Attrition'] = hr_data['Attrition'].map({'Yes': 1, 'No': 0})
hr_data['BusinessTravel'] = hr_data['BusinessTravel'].map({'Non-Travel': 0, 'Travel_Rarely': 1, 'Travel_Frequently': 2})
hr_data['DistanceFromHome'] = hr_data['DistanceFromHome'].astype(int)
hr_data['Education'] = hr_data['Education'].astype('category')
hr_data['EducationField'] = hr_data['EducationField'].astype('category')
hr_data['EmployeeID'] = hr_data['EmployeeID'].astype(int)
hr_data['Gender'] = hr_data['Gender'].map({'Male': 1, 'Female': 0})
hr_data['JobLevel'] = hr_data['JobLevel'].astype(int)
hr_data['JobRole'] = hr_data['JobRole'].astype('category')
hr_data['MaritalStatus'] = hr_data['MaritalStatus'].map({'Single': 0, 'Married': 1, 'Divorced': 2})
hr_data['MonthlyIncome'] = hr_data['MonthlyIncome'].astype(float)
hr_data['NumCompaniesWorked'] = hr_data['NumCompaniesWorked'].astype(int)
hr_data['PercentSalaryHike'] = hr_data['PercentSalaryHike'].astype(float) / 100
hr_data['StockOptionLevel'] = hr_data['StockOptionLevel'].astype(int)
hr_data['TotalWorkingYears'] = hr_data['TotalWorkingYears'].astype(int)
hr_data['TrainingTimesLastYear'] = hr_data['TrainingTimesLastYear'].astype(int)
hr_data['YearsAtCompany'] = hr_data['YearsAtCompany'].astype(int)
hr_data['YearsSinceLastPromotion'] = hr_data['YearsSinceLastPromotion'].astype(int)
hr_data['YearsWithCurrManager'] = hr_data['YearsWithCurrManager'].astype(int)
hr_data['JobInvolvement'] = hr_data['JobInvolvement'].astype('category')
hr_data['PerformanceRating'] = hr_data['PerformanceRating'].astype('category')
hr_data['EnvironmentSatisfaction'] = hr_data['EnvironmentSatisfaction'].astype('category')
hr_data['JobSatisfaction'] = hr_data['JobSatisfaction'].astype('category')
hr_data['WorkLifeBalance'] = hr_data['WorkLifeBalance'].astype('category')

# Sauvegarder les données fusionnées
hr_data.to_csv('merged_data.csv', index=False)

C:\Users\mathy\AppData\Local\Temp\ipykernel_70120\2797232800.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      NaT
1      NaT
2      NaT
3      NaT
4      NaT
        ..
4405   NaT
4406   NaT
4407   NaT
4408   NaT
4409   NaT
Name: 2015-01-01, Length: 4410, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  in_time_data.iloc[:, 1:] = in_time_data.iloc[:, 1:].apply(pd.to_datetime)
C:\Users\mathy\AppData\Local\Temp\ipykernel_70120\2797232800.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      NaT
1      NaT
2      NaT
3      NaT
4      NaT
        ..
4405   NaT
4406   NaT
4407   NaT
4408   NaT
4409   NaT
Name: 2015-01-14, Length: 4410, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  in_time_data.iloc[:, 1

In [10]:
# Verification du type de chaque colonne
#print(hr_data.dtypes)

# Voir les donnees manquantes
#print(hr_data.isnull().sum())
#print(survey_data.isnull().sum())
#print(manager_data.isnull().sum())
# Normalisation des données
categorical_columns = ['Departement', 'EducationField', 'JobRole']
binary_columns = ['Attrition', 'Gender']
numerical_columns = hr_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_columns = [col for col in numerical_columns if col not in categorical_columns + binary_columns]

scaler = MinMaxScaler()
hr_data[numerical_columns] = scaler.fit_transform(hr_data[numerical_columns])

hr_data.to_csv('merged_data_normalized.csv', index=False)

print(hr_data.head())



   Age  Attrition  BusinessTravel              Department  DistanceFromHome  \
0   51          0             0.5                   Sales                 6   
1   31          1             1.0  Research & Development                10   
2   32          0             1.0  Research & Development                17   
3   38          0             0.0  Research & Development                 2   
4   32          0             0.5  Research & Development                10   

  Education EducationField  EmployeeID  Gender  JobLevel  ...  \
0         2  Life Sciences           1       0         1  ...   
1         1  Life Sciences           2       0         1  ...   
2         4          Other           3       1         4  ...   
3         5  Life Sciences           4       1         3  ...   
4         1        Medical           5       1         1  ...   

  TotalWorkingYears  TrainingTimesLastYear  YearsAtCompany  \
0                 1                      6               1   
1         

In [11]:
# Analyse univariée
# Afficher les répartitions des données

import pandas as pd
# enlever les warnings
import warnings
warnings.filterwarnings('ignore')
# Charger les données fusionnées
hr_data = pd.read_csv('merged_data.csv')

# Créer un dictionnaire pour stocker les statistiques
statistics = []

# Calculer les statistiques pour chaque colonne
def calculate_statistics(data, statistics):
    for column in data.columns:
        col_data = data[column]
        if col_data.dtypes == 'object':  # Colonnes catégorielles
            value_counts = col_data.value_counts().to_dict()
            statistics.append({
                'Column': column,
                'Type': 'Categorical',
                'Value Counts': value_counts
            })
        else:  # Colonnes numériques
            statistics.append({
                'Column': column,
                'Type': 'Numeric',
                'Mean': col_data.mean(),
                'Median': col_data.median(),
                'StdDev': col_data.std(),
                'Min': col_data.min(),
                'Max': col_data.max(),
                'Unique Values': col_data.nunique()
            })

# Appeler la fonction de calcul des statistiques
calculate_statistics(hr_data, statistics)

# Convertir les statistiques en DataFrame pour l'export
statistics_df = pd.DataFrame(statistics)

# Sauvegarder les statistiques dans un fichier CSV
statistics_df.to_csv('statistics-data.csv', index=False)

# Afficher les statistiques dans la console
# print(statistics_df)


In [12]:
# Caractéristiques des employés qui ont quitté l'entreprise
# Séparer les employés ayant quitté l'entreprise
attrition_yes = hr_data[hr_data['Attrition'] == 'Yes']
# Calculer les statistiques pour les employés ayant quitté l'entreprise
statistics_attrition = []
calculate_statistics(attrition_yes, statistics_attrition)
statistics_attrition_df = pd.DataFrame(statistics_attrition)
statistics_attrition_df.to_csv('statistics-attrition.csv', index=False)
#print(statistics_attrition_df)

In [ ]:
# MODELE DE PREDICTION

